In [2]:
import os
from openai import OpenAI
import pandas as pd
import numpy as np
from datasets import load_metric
import evaluate
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
rouge_metric = load_metric('rouge')

C:\Users\Dell\AppData\Local\Temp\ipykernel_28764\3964258435.py:9: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric('rouge')
C:\Users\Dell\AppData\Roaming\Python\Python311\site-packages\datasets\load.py:756: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [3]:
AZURE_OPENAI_KEY = "[REPLACE KEY ]"
AZURE_OPENAI_ENDPOINT = "https://tmap-openai.openai.azure.com"

In [4]:
import os
from openai import AzureOpenAI

client = AzureOpenAI(
  azure_endpoint = "https://tmap-openai.openai.azure.com", 
  api_key="[REPLACE KEY ]",  
  api_version="2023-09-01-preview",
)

In [5]:
#print(client.models.list())

### GPT - ATS - Prompt Tuning Experiements

In [6]:
sample_test_df = pd.read_csv('./input/test.csv', nrows=100)
sample_test_df = sample_test_df.sample(20)

In [7]:
def generate_GPT_ATS_Summary(record, model, temp, max_tokens):
    summary = ""
    
    abs_pmt = f"""
                Your task is to create a concise, factual summary, 
                
                by selecting and combining key sentences from  the original text. 
                
                Text is delimited by triple backticks. 

                TEXT: ```{record['article']}```
            """
     try:
        response = client.chat.completions.create(
        model=model, 
        messages=[
            {"role": "system", "content": "You are a LLM trained by OpenAI."},
            {"role": "user", "content": abs_pmt}

        ],
        max_tokens = max_tokens,
        temperature = temp,
        n=1,        
        )
        summary = response.choices[0].message.content
    except Exception:
        print(Exception)
        summary = "ERROR"
    
    return summary

In [8]:

def generateAbsBaselineComplexPrompt(text):
    summary = ""
    
    abs_pmt = f"""
                As a LLM trained by OpenAI, create a concise and factualsummary, 
                
                by selecting and combining key sentences from the original text,
                
                while adhering to these guidelines:
                
                1.Craft a summary that is detailed, in-depth, while maintaining clarity and conciseness.
                
                2.Incorporate main ideas, incorporate essential information and focus on critical aspects.
                
                3.Rely strictly on the provided text, without including external information.
                
                Text is delimited by triple backticks TEXT: ```{text}```
                """
   
    
    #create a concise summary by selecting and combining key sentences from the original text : {text}"
    try:
        response = client.chat.completions.create(
        model="gpt4", # model = "deployment_name".
        messages=[
            {"role": "system", "content": "You are a LLM trained by OpenAI."},
            {"role": "user", "content": abs_pmt}

        ])
        summary = response.choices[0].message.content
    except Exception:
        print(Exception)
        summary = "ERROR"
    
    return summary





In [9]:
sample_test_df['gpt4_ats_low_temp_30_words'] = sample_test_df.apply(lambda rec: generate_GPT_ATS_Summary(rec, 'gpt4', 0, 30), axis=1)
sample_test_df['gpt4_ats_med_temp_30_words'] = sample_test_df.apply(lambda rec: generate_GPT_ATS_Summary(rec, 'gpt4', 1, 30), axis=1)
sample_test_df['gpt4_ats_high_temp_30_words'] = sample_test_df.apply(lambda rec: generate_GPT_ATS_Summary(rec, 'gpt4', 2, 30), axis=1)

sample_test_df['gpt35_ats_low_temp_30_words'] = sample_test_df.apply(lambda rec: generate_GPT_ATS_Summary(rec, 'gpt35', 0, 30), axis=1)
sample_test_df['gpt35_ats_med_temp_30_words'] = sample_test_df.apply(lambda rec: generate_GPT_ATS_Summary(rec, 'gpt35', 1, 30), axis=1)
sample_test_df['gpt35_ats_high_temp_30_words'] = sample_test_df.apply(lambda rec: generate_GPT_ATS_Summary(rec, 'gpt35', 2, 30), axis=1)

sample_test_df['gpt4_ats_low_temp_60_words'] = sample_test_df.apply(lambda rec: generate_GPT_ATS_Summary(rec, 'gpt4', 0, 60), axis=1)
sample_test_df['gpt4_ats_med_temp_60_words'] = sample_test_df.apply(lambda rec: generate_GPT_ATS_Summary(rec, 'gpt4', 1, 60), axis=1)
sample_test_df['gpt4_ats_high_temp_60_words'] = sample_test_df.apply(lambda rec: generate_GPT_ATS_Summary(rec, 'gpt4', 2, 60), axis=1)

sample_test_df['gpt35_ats_low_temp_60_words'] = sample_test_df.apply(lambda rec: generate_GPT_ATS_Summary(rec, 'gpt35', 0, 60), axis=1)
sample_test_df['gpt35_ats_med_temp_60_words'] = sample_test_df.apply(lambda rec: generate_GPT_ATS_Summary(rec, 'gpt35', 1, 60), axis=1)
sample_test_df['gpt35_ats_high_temp_60_words'] = sample_test_df.apply(lambda rec: generate_GPT_ATS_Summary(rec, 'gpt35', 2, 60), axis=1)

sample_test_df['gpt4_ats_low_temp_90_words'] = sample_test_df.apply(lambda rec: generate_GPT_ATS_Summary(rec, 'gpt4', 0, 90), axis=1)
sample_test_df['gpt4_ats_med_temp_90_words'] = sample_test_df.apply(lambda rec: generate_GPT_ATS_Summary(rec, 'gpt4', 1, 90), axis=1)
sample_test_df['gpt4_ats_high_temp_90_words'] = sample_test_df.apply(lambda rec: generate_GPT_ATS_Summary(rec, 'gpt4', 2, 90), axis=1)

sample_test_df['gpt35_ats_low_temp_90_words'] = sample_test_df.apply(lambda rec: generate_GPT_ATS_Summary(rec, 'gpt35', 0, 90), axis=1)
sample_test_df['gpt35_ats_med_temp_90_words'] = sample_test_df.apply(lambda rec: generate_GPT_ATS_Summary(rec, 'gpt35', 1, 90), axis=1)
sample_test_df['gpt35_ats_high_temp_90_words'] = sample_test_df.apply(lambda rec: generate_GPT_ATS_Summary(rec, 'gpt35', 2, 90), axis=1)

sample_test_df['gpt4_ats_low_temp_120_words'] = sample_test_df.apply(lambda rec: generate_GPT_ATS_Summary(rec, 'gpt4', 0, 120), axis=1)
sample_test_df['gpt4_ats_med_temp_120_words'] = sample_test_df.apply(lambda rec: generate_GPT_ATS_Summary(rec, 'gpt4', 1, 120), axis=1)
sample_test_df['gpt4_ats_high_temp_120_words'] = sample_test_df.apply(lambda rec: generate_GPT_ATS_Summary(rec, 'gpt4', 2, 120), axis=1)

sample_test_df['gpt35_ats_low_temp_120_words'] = sample_test_df.apply(lambda rec: generate_GPT_ATS_Summary(rec, 'gpt35', 0, 120), axis=1)
sample_test_df['gpt35_ats_med_temp_120_words'] = sample_test_df.apply(lambda rec: generate_GPT_ATS_Summary(rec, 'gpt35', 1, 120), axis=1)
sample_test_df['gpt35_ats_high_temp_120_words'] = sample_test_df.apply(lambda rec: generate_GPT_ATS_Summary(rec, 'gpt35', 2, 120), axis=1)



<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>
<class 'Exception'>


In [10]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

models = [
          
          "gpt4_ats_low_temp_30_words",  "gpt4_ats_med_temp_30_words",  "gpt4_ats_high_temp_30_words",
          "gpt35_ats_low_temp_30_words",  "gpt35_ats_med_temp_30_words",  "gpt35_ats_high_temp_30_words",
          
          "gpt4_ats_low_temp_60_words",  "gpt4_ats_med_temp_60_words",  "gpt4_ats_high_temp_60_words",
          "gpt35_ats_low_temp_60_words",  "gpt35_ats_med_temp_60_words",  "gpt35_ats_high_temp_60_words",
          
          "gpt4_ats_low_temp_90_words",  "gpt4_ats_med_temp_90_words",  "gpt4_ats_high_temp_90_words",
          "gpt35_ats_low_temp_90_words",  "gpt35_ats_med_temp_90_words",  "gpt35_ats_high_temp_90_words",
          
          "gpt4_ats_low_temp_120_words",  "gpt4_ats_med_temp_120_words",  "gpt4_ats_high_temp_120_words",
          "gpt35_ats_low_temp_120_words",  "gpt35_ats_med_temp_120_words",  "gpt35_ats_high_temp_120_words",
         ]


records = []

for model_name in models:
    rouge_metric.add(prediction = sample_test_df[model_name], reference = sample_test_df['highlights'])
    score = rouge_metric.compute()
    rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )
    print('rouge_dict ', rouge_dict )
    records.append(rouge_dict)

metrics_df = pd.DataFrame.from_records(records, index = models )
metrics_df


rouge_dict  {'rouge1': 0.4053058216654385, 'rouge2': 0.1107011070110701, 'rougeL': 0.210759027266028, 'rougeLsum': 0.210759027266028}
rouge_dict  {'rouge1': 0.37278106508875747, 'rouge2': 0.08592592592592592, 'rougeL': 0.18491124260355035, 'rougeLsum': 0.18491124260355035}
rouge_dict  {'rouge1': 0.2718589272593681, 'rouge2': 0.05003679175864606, 'rougeL': 0.13225569434239529, 'rougeLsum': 0.13225569434239529}
rouge_dict  {'rouge1': 0.41249092229484385, 'rouge2': 0.1090909090909091, 'rougeL': 0.19607843137254902, 'rougeLsum': 0.19607843137254902}
rouge_dict  {'rouge1': 0.3932002956393201, 'rouge2': 0.08882309400444115, 'rougeL': 0.17294900221729487, 'rougeLsum': 0.17294900221729487}
rouge_dict  {'rouge1': 0.11596385542168675, 'rouge2': 0.016591251885369532, 'rougeL': 0.05722891566265059, 'rougeLsum': 0.05722891566265059}
rouge_dict  {'rouge1': 0.444179104477612, 'rouge2': 0.1123729826658697, 'rougeL': 0.19940298507462687, 'rougeLsum': 0.19940298507462687}
rouge_dict  {'rouge1': 0.447337

,rouge1,rouge2,rougeL,rougeLsum
gpt4_ats_low_temp_30_words,0.405306,0.110701,0.210759,0.210759
gpt4_ats_med_temp_30_words,0.372781,0.085926,0.184911,0.184911
gpt4_ats_high_temp_30_words,0.271859,0.050037,0.132256,0.132256
gpt35_ats_low_temp_30_words,0.412491,0.109091,0.196078,0.196078
gpt35_ats_med_temp_30_words,0.393200,0.088823,0.172949,0.172949
gpt35_ats_high_temp_30_words,0.115964,0.016591,0.057229,0.057229
gpt4_ats_low_temp_60_words,0.444179,0.112373,0.199403,0.199403
gpt4_ats_med_temp_60_words,0.447337,0.107820,0.182249,0.182249
gpt4_ats_high_temp_60_words,0.187683,0.032883,0.089150,0.089150
gpt35_ats_low_temp_60_words,0.502570,0.149800,0.210166,0.210166


In [11]:
metrics_df.to_csv('./output/GPT_ATS_PT_Metrics.csv')

### GPT - ATS - Few Shots Inferencing Experiements

In [12]:
sample_test_df = pd.read_csv('./input/test.csv', nrows=100)

In [13]:
# Computation of Length of article and hughlights
def article_len(row):
    return len(row['article'].split())

def highlights_len(row):
    return len(row['highlights'].split())


sample_test_df['article_len'] = sample_test_df.apply(lambda r: article_len(r), axis= 1)
sample_test_df['highlights_len'] = sample_test_df.apply(lambda r: highlights_len(r), axis= 1)


In [14]:
sorted_by_article_size_df = sample_test_df.sort_values('article_len')

In [15]:
sample_test_df = sample_test_df[:20]

In [16]:
def generate_GPT_Abs_1_Shot(record, model, temp, max_tokens):
    summary = ""
    abs_pmt = f"""
                Your task is to create a concise, factual summary, 
                
                by selecting and combining key sentences from  the original text. 
                
                Text is delimited by triple backticks. 

                TEXT: ```{record['article']}```
            """
    
    #create a concise summary by selecting and combining key sentences from the original text : {text}"
    try:
        response = client.chat.completions.create(
        model=model, # model = "deployment_name".
        messages=[
            {"role": "system", "content": "You are a LLM trained by OpenAI."},
            {"role":"user","content":sorted_by_article_size_df.iloc[0]['article']},
            {"role":"assistant","content":sorted_by_article_size_df.iloc[0]['highlights']},
            {"role": "user", "content": abs_pmt},
            ],
        max_tokens = max_tokens,
        temperature = temp, #top_p=.9
        n=1,
        )
        summary = response.choices[0].message.content
    except Exception:
        print(Exception)
        summary = "ERROR"
    
    return summary

def generate_GPT_Abs_3_Shots(record, model, temp, max_tokens):
    summary = ""
    
    abs_pmt = f"""
                Your task is to create a concise, factual summary, 
                
                by selecting and combining key sentences from  the original text. 
                
                Text is delimited by triple backticks. 

                TEXT: ```{record['article']}```
            """
    try:
        response = client.chat.completions.create(
        model=model, 
        messages=[
            {"role": "system", "content": "You are a LLM trained by OpenAI."},

            {"role":"user","content":sorted_by_article_size_df.iloc[0]['article']},
            {"role":"assistant","content":sorted_by_article_size_df.iloc[0]['highlights']},
            
            {"role":"user","content":sorted_by_article_size_df.iloc[1]['article']},
            {"role":"assistant","content":sorted_by_article_size_df.iloc[1]['highlights']},

            {"role":"user","content":sorted_by_article_size_df.iloc[2]['article']},
            {"role":"assistant","content":sorted_by_article_size_df.iloc[2]['highlights']},
            
            {"role": "user", "content": abs_pmt},
            
        ],
        max_tokens = max_tokens,
        temperature = temp,
        n=1,
        )
        summary = response.choices[0].message.content
    except Exception as e:
        print(e)
        summary = "ERROR"
    
    return summary

def generate_GPT_Abs_5_Shots(record, model, temp, max_tokens):
    summary = ""
       
    abs_pmt = f"""
                Your task is to create a concise, factual summary, 

                by selecting and combining key sentences from  the original text. 

                Text is delimited by triple backticks. 

                TEXT: ```{record['article']}```
            """
    #print('ARTICLE ', record['article'][:200])
        #create a concise summary by selecting and combining key sentences from the original text : {text}"
    try:
        response = client.chat.completions.create(
            model= model, # model = "deployment_name".
            messages=[
                {"role": "system", "content": "You are a LLM trained by OpenAI."},

                {"role":"user","content":sorted_by_article_size_df.iloc[0]['article']},
                {"role":"assistant","content":sorted_by_article_size_df.iloc[0]['highlights']},

                {"role":"user","content":sorted_by_article_size_df.iloc[1]['article']},
                {"role":"assistant","content":sorted_by_article_size_df.iloc[1]['highlights']},

                {"role":"user","content":sorted_by_article_size_df.iloc[2]['article']},
                {"role":"assistant","content":sorted_by_article_size_df.iloc[2]['highlights']},

                {"role":"user","content":sorted_by_article_size_df.iloc[3]['article']},
                {"role":"assistant","content":sorted_by_article_size_df.iloc[3]['highlights']},

                {"role":"user","content":sorted_by_article_size_df.iloc[4]['article']},
                {"role":"assistant","content":sorted_by_article_size_df.iloc[4]['highlights']},

                {"role": "user", "content": abs_pmt},

            ],
            max_tokens = max_tokens,
            temperature = temp, #top_p=.9
            n=1,
            )
        #print("Summary - ", response.choices[0].message.content)
        summary = response.choices[0].message.content
    except Exception as e:
        print(e)
        summary = "ERROR"
  
    return summary

In [17]:
#generate_GPT_Abs_5_Shots(sample_test_df.iloc[0], 'gpt4', 2, 60)

In [18]:

# Apply the function to each row of the dataframe
sample_test_df['gpt4-abs-5_shots_low_temp_60_words'] = sample_test_df.apply(lambda rec: generate_GPT_Abs_5_Shots(rec, 'gpt4', 0, 60), axis=1)
sample_test_df['gpt4-abs-5_shots_med_temp_60_words'] = sample_test_df.apply(lambda rec: generate_GPT_Abs_5_Shots(rec, 'gpt4', 1, 60), axis=1)
sample_test_df['gpt4-abs-5_shots_high_temp_60_words'] = sample_test_df.apply(lambda rec: generate_GPT_Abs_5_Shots(rec, 'gpt4', 2, 60), axis=1)

sample_test_df['gpt35-abs-5_shots_low_temp_60_words'] = sample_test_df.apply(lambda rec: generate_GPT_Abs_5_Shots(rec, 'gpt35', 0, 60), axis=1)
sample_test_df['gpt35-abs-5_shots_med_temp_60_words'] = sample_test_df.apply(lambda rec: generate_GPT_Abs_5_Shots(rec, 'gpt35', 1, 60), axis=1)
sample_test_df['gpt35-abs-5_shots_high_temp_60_words'] = sample_test_df.apply(lambda rec: generate_GPT_Abs_5_Shots(rec, 'gpt35', 2, 60), axis=1)

sample_test_df['gpt4-abs-3_shots_low_temp_60_words'] = sample_test_df.apply(lambda rec: generate_GPT_Abs_3_Shots(rec, 'gpt4', 0, 60), axis=1)
sample_test_df['gpt4-abs-3_shots_med_temp_60_words'] = sample_test_df.apply(lambda rec: generate_GPT_Abs_3_Shots(rec, 'gpt4', 1, 60), axis=1)
sample_test_df['gpt4-abs-3_shots_high_temp_60_words'] = sample_test_df.apply(lambda rec: generate_GPT_Abs_3_Shots(rec, 'gpt4', 2, 60), axis=1)

sample_test_df['gpt35-abs-3_shots_low_temp_60_words'] = sample_test_df.apply(lambda rec: generate_GPT_Abs_3_Shots(rec, 'gpt35', 0, 60), axis=1)
sample_test_df['gpt35-abs-3_shots_med_temp_60_words'] = sample_test_df.apply(lambda rec: generate_GPT_Abs_3_Shots(rec, 'gpt35', 1, 60), axis=1)
sample_test_df['gpt35-abs-3_shots_high_temp_60_words'] = sample_test_df.apply(lambda rec: generate_GPT_Abs_3_Shots(rec, 'gpt35', 2, 60), axis=1)

sample_test_df['gpt4-abs-1_shot_low_temp_60_words'] = sample_test_df.apply(lambda rec: generate_GPT_Abs_1_Shot(rec, 'gpt4', 0, 60), axis=1)
sample_test_df['gpt4-abs-1_shot_med_temp_60_words'] = sample_test_df.apply(lambda rec: generate_GPT_Abs_1_Shot(rec, 'gpt4', 1, 60), axis=1)
sample_test_df['gpt4-abs-1_shot_high_temp_60_words'] = sample_test_df.apply(lambda rec: generate_GPT_Abs_1_Shot(rec, 'gpt4', 2, 60), axis=1)

sample_test_df['gpt35-abs-1_shot_low_temp_60_words'] = sample_test_df.apply(lambda rec: generate_GPT_Abs_1_Shot(rec, 'gpt35', 0, 60), axis=1)
sample_test_df['gpt35-abs-1_shot_med_temp_60_words'] = sample_test_df.apply(lambda rec: generate_GPT_Abs_1_Shot(rec, 'gpt35', 1, 60), axis=1)
sample_test_df['gpt35-abs-1_shot_high_temp_60_words'] = sample_test_df.apply(lambda rec: generate_GPT_Abs_1_Shot(rec, 'gpt35', 2, 60), axis=1)


Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': True, 'severity': 'medium'}}}}}
Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param

In [19]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
models = [

          'gpt4-abs-5_shots_low_temp_60_words', 
          'gpt4-abs-5_shots_med_temp_60_words' , 
          'gpt4-abs-5_shots_high_temp_60_words',
          'gpt35-abs-5_shots_low_temp_60_words', 
          'gpt35-abs-5_shots_med_temp_60_words', 
          'gpt35-abs-5_shots_high_temp_60_words',

          'gpt4-abs-3_shots_low_temp_60_words', 
          'gpt4-abs-3_shots_med_temp_60_words' , 
          'gpt4-abs-3_shots_high_temp_60_words',
          'gpt35-abs-3_shots_low_temp_60_words', 
          'gpt35-abs-3_shots_med_temp_60_words', 
          'gpt35-abs-3_shots_high_temp_60_words',
    
          'gpt4-abs-1_shot_low_temp_60_words', 
          'gpt4-abs-1_shot_med_temp_60_words' , 
          'gpt4-abs-1_shot_high_temp_60_words',
          'gpt35-abs-1_shot_low_temp_60_words', 
          'gpt35-abs-1_shot_med_temp_60_words', 
          'gpt35-abs-1_shot_high_temp_60_words',
         ]

records = []

for model_name in models:
    rouge_metric.add(prediction = sample_test_df[model_name], reference = sample_test_df['highlights'])
    score = rouge_metric.compute()
    rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )
    print('rouge_dict ', rouge_dict )
    records.append(rouge_dict)

metrics_df = pd.DataFrame.from_records(records, index = models )
metrics_df

rouge_dict  {'rouge1': 0.48203939745075325, 'rouge2': 0.14849187935034802, 'rougeL': 0.22363847045191196, 'rougeLsum': 0.22363847045191196}
rouge_dict  {'rouge1': 0.47449584816132867, 'rouge2': 0.14489311163895488, 'rougeL': 0.21352313167259784, 'rougeLsum': 0.21352313167259784}
rouge_dict  {'rouge1': 0.2779069767441861, 'rouge2': 0.0710128055878929, 'rougeL': 0.13488372093023257, 'rougeLsum': 0.13488372093023257}
rouge_dict  {'rouge1': 0.5122646891043925, 'rouge2': 0.1850371216447744, 'rougeL': 0.257843696520251, 'rougeLsum': 0.257843696520251}
rouge_dict  {'rouge1': 0.5200892857142857, 'rouge2': 0.13743016759776536, 'rougeL': 0.23325892857142855, 'rougeLsum': 0.23325892857142855}
rouge_dict  {'rouge1': 0.07082833133253301, 'rouge2': 0.007211538461538461, 'rougeL': 0.03361344537815126, 'rougeLsum': 0.03361344537815126}
rouge_dict  {'rouge1': 0.47884057971014493, 'rouge2': 0.15786419036564134, 'rougeL': 0.22724637681159424, 'rougeLsum': 0.22724637681159424}
rouge_dict  {'rouge1': 0.493

,rouge1,rouge2,rougeL,rougeLsum
gpt4-abs-5_shots_low_temp_60_words,0.482039,0.148492,0.223638,0.223638
gpt4-abs-5_shots_med_temp_60_words,0.474496,0.144893,0.213523,0.213523
gpt4-abs-5_shots_high_temp_60_words,0.277907,0.071013,0.134884,0.134884
gpt35-abs-5_shots_low_temp_60_words,0.512265,0.185037,0.257844,0.257844
gpt35-abs-5_shots_med_temp_60_words,0.520089,0.137430,0.233259,0.233259
gpt35-abs-5_shots_high_temp_60_words,0.070828,0.007212,0.033613,0.033613
gpt4-abs-3_shots_low_temp_60_words,0.478841,0.157864,0.227246,0.227246
gpt4-abs-3_shots_med_temp_60_words,0.493392,0.173098,0.240088,0.240088
gpt4-abs-3_shots_high_temp_60_words,0.268508,0.067478,0.119337,0.119337
gpt35-abs-3_shots_low_temp_60_words,0.531588,0.184615,0.260672,0.260672


In [20]:
metrics_df.to_csv("./output/GPT_ATS_FS_Metrics.csv")

### GPT ETS Prompt Tuning

In [21]:
sample_test_df = pd.read_csv('./input/test.csv', nrows=100)
sample_test_df = sample_test_df[:20]

In [22]:
def generate_GPT_Ext_Summary(record, model, temp, max_tokens):
    summary = ""
    
    ext_pmt = f"""
                Your task is to Generate extractive summary from the original text, 
                
                by identifying 4-5 key sentences from the original text and 
                
                DO NOT rewrite or paraphrase the content.
                
                Text is delimited by triple backticks. 

                TEXT: ```{record['article']}```
            """
    try:
        response = client.chat.completions.create(
        model=model, 
        messages=[
            {"role": "system", "content": "You are a LLM trained by OpenAI."},
            {"role": "user", "content": ext_pmt}

        ],
        max_tokens = max_tokens,
        temperature = temp,
        n=1,
        )
        summary = response.choices[0].message.content
    except Exception as e:
        print(e)
        summary = "ERROR"
    
    return summary

In [23]:
# Apply the function to each row of the dataframe

sample_test_df['gpt4_ets_low_temp_30_words'] = sample_test_df.apply(lambda rec: generate_GPT_Ext_Summary(rec, 'gpt4', 0, 30), axis=1)
sample_test_df['gpt4_ets_med_temp_30_words'] = sample_test_df.apply(lambda rec: generate_GPT_Ext_Summary(rec, 'gpt4', 1, 30), axis=1)
sample_test_df['gpt4-ets_high_temp_30_words'] = sample_test_df.apply(lambda rec: generate_GPT_Ext_Summary(rec, 'gpt4', 2, 30), axis=1)

sample_test_df['gpt35_ets_low_temp_30_words'] = sample_test_df.apply(lambda rec: generate_GPT_Ext_Summary(rec, 'gpt35', 0, 30), axis=1)
sample_test_df['gpt35_ets_med_temp_30_words'] = sample_test_df.apply(lambda rec: generate_GPT_Ext_Summary(rec, 'gpt35', 1, 30), axis=1)
sample_test_df['gpt35-ets_high_temp_30_words'] = sample_test_df.apply(lambda rec: generate_GPT_Ext_Summary(rec, 'gpt35', 2, 30), axis=1)

sample_test_df['gpt4_ets_low_temp_60_words'] = sample_test_df.apply(lambda rec: generate_GPT_Ext_Summary(rec, 'gpt4', 0, 60), axis=1)
sample_test_df['gpt4_ets_med_temp_60_words'] = sample_test_df.apply(lambda rec: generate_GPT_Ext_Summary(rec, 'gpt4', 1, 60), axis=1)
sample_test_df['gpt4-ets_high_temp_60_words'] = sample_test_df.apply(lambda rec: generate_GPT_Ext_Summary(rec, 'gpt4', 2, 60), axis=1)

sample_test_df['gpt35_ets_low_temp_60_words'] = sample_test_df.apply(lambda rec: generate_GPT_Ext_Summary(rec, 'gpt35', 0, 60), axis=1)
sample_test_df['gpt35_ets_med_temp_60_words'] = sample_test_df.apply(lambda rec: generate_GPT_Ext_Summary(rec, 'gpt35', 1, 60), axis=1)
sample_test_df['gpt35-ets_high_temp_60_words'] = sample_test_df.apply(lambda rec: generate_GPT_Ext_Summary(rec, 'gpt35', 2, 60), axis=1)

sample_test_df['gpt4_ets_low_temp_90_words'] = sample_test_df.apply(lambda rec: generate_GPT_Ext_Summary(rec, 'gpt4', 0, 90), axis=1)
sample_test_df['gpt4_ets_med_temp_90_words'] = sample_test_df.apply(lambda rec: generate_GPT_Ext_Summary(rec, 'gpt4', 1, 90), axis=1)
sample_test_df['gpt4-ets_high_temp_90_words'] = sample_test_df.apply(lambda rec: generate_GPT_Ext_Summary(rec, 'gpt4', 2, 90), axis=1)

sample_test_df['gpt35_ets_low_temp_90_words'] = sample_test_df.apply(lambda rec: generate_GPT_Ext_Summary(rec, 'gpt35', 0, 90), axis=1)
sample_test_df['gpt35_ets_med_temp_90_words'] = sample_test_df.apply(lambda rec: generate_GPT_Ext_Summary(rec, 'gpt35', 1, 90), axis=1)
sample_test_df['gpt35-ets_high_temp_90_words'] = sample_test_df.apply(lambda rec: generate_GPT_Ext_Summary(rec, 'gpt35', 2, 90), axis=1)

sample_test_df['gpt4_ets_low_temp_120_words'] = sample_test_df.apply(lambda rec: generate_GPT_Ext_Summary(rec, 'gpt4', 0, 120), axis=1)
sample_test_df['gpt4_ets_med_temp_120_words'] = sample_test_df.apply(lambda rec: generate_GPT_Ext_Summary(rec, 'gpt4', 1, 120), axis=1)
sample_test_df['gpt4-ets_high_temp_120_words'] = sample_test_df.apply(lambda rec: generate_GPT_Ext_Summary(rec, 'gpt4', 2, 120), axis=1)

sample_test_df['gpt35_ets_low_temp_120_words'] = sample_test_df.apply(lambda rec: generate_GPT_Ext_Summary(rec, 'gpt35', 0, 120), axis=1)
sample_test_df['gpt35_ets_med_temp_120_words'] = sample_test_df.apply(lambda rec: generate_GPT_Ext_Summary(rec, 'gpt35', 1, 120), axis=1)
sample_test_df['gpt35-ets_high_temp_120_words'] = sample_test_df.apply(lambda rec: generate_GPT_Ext_Summary(rec, 'gpt35', 2, 120), axis=1)


Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': True, 'severity': 'medium'}}}}}
Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param

Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': True, 'severity': 'medium'}}}}}
Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param

In [24]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
models = [

          'gpt4_ets_low_temp_30_words', 
          'gpt4_ets_med_temp_30_words' , 
          'gpt4-ets_high_temp_30_words',
          'gpt35_ets_low_temp_30_words', 
          'gpt35_ets_med_temp_30_words', 
          'gpt35-ets_high_temp_30_words',
    
          'gpt4_ets_low_temp_60_words', 
          'gpt4_ets_med_temp_60_words' , 
          'gpt4-ets_high_temp_60_words',
          'gpt35_ets_low_temp_60_words', 
          'gpt35_ets_med_temp_60_words', 
          'gpt35-ets_high_temp_60_words',

          'gpt4_ets_low_temp_90_words', 
          'gpt4_ets_med_temp_90_words' , 
          'gpt4-ets_high_temp_90_words',
          'gpt35_ets_low_temp_90_words', 
          'gpt35_ets_med_temp_90_words', 
          'gpt35-ets_high_temp_90_words',

          'gpt4_ets_low_temp_120_words', 
          'gpt4_ets_med_temp_120_words' , 
          'gpt4-ets_high_temp_120_words',
          'gpt35_ets_low_temp_120_words', 
          'gpt35_ets_med_temp_120_words', 
          'gpt35-ets_high_temp_120_words',
         ]

records = []

for model_name in models:
    rouge_metric.add(prediction = sample_test_df[model_name], reference = sample_test_df['highlights'])
    score = rouge_metric.compute()
    rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )
    print('rouge_dict ', rouge_dict )
    records.append(rouge_dict)

metrics_df = pd.DataFrame.from_records(records, index = models )
metrics_df

rouge_dict  {'rouge1': 0.3927068723702665, 'rouge2': 0.1348314606741573, 'rougeL': 0.2187938288920056, 'rougeLsum': 0.2187938288920056}
rouge_dict  {'rouge1': 0.38450704225352106, 'rouge2': 0.1086036671368124, 'rougeL': 0.19999999999999998, 'rougeLsum': 0.19999999999999998}
rouge_dict  {'rouge1': 0.35090252707581226, 'rouge2': 0.11713665943600866, 'rougeL': 0.1891696750902527, 'rougeLsum': 0.1891696750902527}
rouge_dict  {'rouge1': 0.4206848357791754, 'rouge2': 0.1497550734779566, 'rougeL': 0.2180293501048218, 'rougeLsum': 0.2180293501048218}
rouge_dict  {'rouge1': 0.38461538461538464, 'rouge2': 0.10504201680672269, 'rougeL': 0.20139860139860138, 'rougeLsum': 0.20139860139860138}
rouge_dict  {'rouge1': 0.1457725947521866, 'rouge2': 0.02335766423357664, 'rougeL': 0.07434402332361517, 'rougeLsum': 0.07434402332361517}
rouge_dict  {'rouge1': 0.4985706117781589, 'rouge2': 0.15798511734401832, 'rougeL': 0.23441966838193257, 'rougeLsum': 0.23441966838193257}
rouge_dict  {'rouge1': 0.49199084

,rouge1,rouge2,rougeL,rougeLsum
gpt4_ets_low_temp_30_words,0.392707,0.134831,0.218794,0.218794
gpt4_ets_med_temp_30_words,0.384507,0.108604,0.200000,0.200000
gpt4-ets_high_temp_30_words,0.350903,0.117137,0.189170,0.189170
gpt35_ets_low_temp_30_words,0.420685,0.149755,0.218029,0.218029
gpt35_ets_med_temp_30_words,0.384615,0.105042,0.201399,0.201399
gpt35-ets_high_temp_30_words,0.145773,0.023358,0.074344,0.074344
gpt4_ets_low_temp_60_words,0.498571,0.157985,0.234420,0.234420
gpt4_ets_med_temp_60_words,0.491991,0.160367,0.239130,0.239130
gpt4-ets_high_temp_60_words,0.467218,0.146718,0.217080,0.217080
gpt35_ets_low_temp_60_words,0.491749,0.157489,0.237624,0.237624


In [25]:
metrics_df.to_csv('./output/GPT_ETS_PT_Metrics.csv')

### GPT ETS Few Shots Tuning

In [26]:
sample_test_df = pd.read_csv('./input/test.csv', nrows=100)

In [27]:
# Computation of Length of article and hughlights
def article_len(row):
    return len(row['article'].split())

def highlights_len(row):
    return len(row['highlights'].split())


sample_test_df['article_len'] = sample_test_df.apply(lambda r: article_len(r), axis= 1)
sample_test_df['highlights_len'] = sample_test_df.apply(lambda r: highlights_len(r), axis= 1)

In [28]:
sorted_by_article_size_df = sample_test_df.sort_values('article_len')

In [29]:
sample_test_df = sample_test_df[:20]

In [30]:
def generate_GPT_ETS_1_Shot(record, model, temp, max_tokens):
    summary = ""
    ext_pmt = f"""
                Your task is to Generate extractive summary from the original text, 
                
                by identifying 4-5 key sentences from the original text and 
                
                DO NOT rewrite or paraphrase the content.
                
                Text is delimited by triple backticks. 

                TEXT: ```{record['article']}```
            """
    
    #create a concise summary by selecting and combining key sentences from the original text : {text}"
    try:
        response = client.chat.completions.create(
        model=model, # model = "deployment_name".
        messages=[
            {"role": "system", "content": "You are a LLM trained by OpenAI."},
            {"role":"user","content":sorted_by_article_size_df.iloc[0]['article']},
            {"role":"assistant","content":sorted_by_article_size_df.iloc[0]['highlights']},
            {"role": "user", "content": ext_pmt},
            ],
        max_tokens = max_tokens,
        temperature = temp, #top_p=.9
        n=1,
        )
        summary = response.choices[0].message.content
    except Exception:
        print(Exception)
        summary = "ERROR"
    
    return summary

def generate_GPT_ETS_3_Shots(record, model, temp, max_tokens):
    summary = ""
    
    ext_pmt = f"""
                Your task is to Generate extractive summary from the original text, 
                
                by identifying 4-5 key sentences from the original text and 
                
                DO NOT rewrite or paraphrase the content.
                
                Text is delimited by triple backticks. 

                TEXT: ```{record['article']}```
            """
    
    #create a concise summary by selecting and combining key sentences from the original text : {text}"
    try:
        response = client.chat.completions.create(
        model=model, # model = "deployment_name".
        messages=[
            {"role": "system", "content": "You are a LLM trained by OpenAI."},

            {"role":"user","content":sorted_by_article_size_df.iloc[0]['article']},
            {"role":"assistant","content":sorted_by_article_size_df.iloc[0]['highlights']},
            
            {"role":"user","content":sorted_by_article_size_df.iloc[1]['article']},
            {"role":"assistant","content":sorted_by_article_size_df.iloc[1]['highlights']},

            {"role":"user","content":sorted_by_article_size_df.iloc[2]['article']},
            {"role":"assistant","content":sorted_by_article_size_df.iloc[2]['highlights']},
            
            {"role": "user", "content": ext_pmt},
            
        ],
        max_tokens = max_tokens,
        temperature = temp, #top_p=.9
        n=1,
        )
        summary = response.choices[0].message.content
    except Exception as e:
        print(e)
        summary = "ERROR"
    
    return summary

def generate_GPT_ETS_5_Shots(record, model, temp, max_tokens):
    summary = ""
       
    ext_pmt = f"""
                Your task is to Generate extractive summary from the original text, 
                
                by identifying 4-5 key sentences from the original text and 
                
                DO NOT rewrite or paraphrase the content.
                
                Text is delimited by triple backticks. 

                TEXT: ```{record['article']}```
            """
    #print('ARTICLE ', record['article'][:200])
        #create a concise summary by selecting and combining key sentences from the original text : {text}"
    try:
        response = client.chat.completions.create(
            model= model, # model = "deployment_name".
            messages=[
                {"role": "system", "content": "You are a LLM trained by OpenAI."},

                {"role":"user","content":sorted_by_article_size_df.iloc[0]['article']},
                {"role":"assistant","content":sorted_by_article_size_df.iloc[0]['highlights']},

                {"role":"user","content":sorted_by_article_size_df.iloc[1]['article']},
                {"role":"assistant","content":sorted_by_article_size_df.iloc[1]['highlights']},

                {"role":"user","content":sorted_by_article_size_df.iloc[2]['article']},
                {"role":"assistant","content":sorted_by_article_size_df.iloc[2]['highlights']},

                {"role":"user","content":sorted_by_article_size_df.iloc[3]['article']},
                {"role":"assistant","content":sorted_by_article_size_df.iloc[3]['highlights']},

                {"role":"user","content":sorted_by_article_size_df.iloc[4]['article']},
                {"role":"assistant","content":sorted_by_article_size_df.iloc[4]['highlights']},

                {"role": "user", "content": ext_pmt},

            ],
            max_tokens = max_tokens,
            temperature = temp, 
            top_p= 0.9,
            presence_penalty=0.01,
            n=1,
            )
        #print("Summary - ", response.choices[0].message.content)
        summary = response.choices[0].message.content
    except Exception as e:
        print(e)
        summary = "ERROR"
  
    return summary

In [31]:
# Apply the function to each row of the dataframe
sample_test_df['gpt4-ets-5_shots_low_temp_60_words'] = sample_test_df.apply(lambda rec: generate_GPT_ETS_5_Shots(rec, 'gpt4', 0, 60), axis=1)
sample_test_df['gpt4-ets-5_shots_med_temp_60_words'] = sample_test_df.apply(lambda rec: generate_GPT_ETS_5_Shots(rec, 'gpt4', 1, 60), axis=1)
sample_test_df['gpt4-ets-5_shots_high_temp_60_words'] = sample_test_df.apply(lambda rec: generate_GPT_ETS_5_Shots(rec, 'gpt4', 2, 60), axis=1)

sample_test_df['gpt35-ets-5_shots_low_temp_60_words'] = sample_test_df.apply(lambda rec: generate_GPT_ETS_5_Shots(rec, 'gpt35', 0, 60), axis=1)
sample_test_df['gpt35-ets-5_shots_med_temp_60_words'] = sample_test_df.apply(lambda rec: generate_GPT_ETS_5_Shots(rec, 'gpt35', 1, 60), axis=1)
sample_test_df['gpt35-ets-5_shots_high_temp_60_words'] = sample_test_df.apply(lambda rec: generate_GPT_ETS_5_Shots(rec, 'gpt35', 2, 60), axis=1)

sample_test_df['gpt4-ets-3_shots_low_temp_60_words'] = sample_test_df.apply(lambda rec: generate_GPT_ETS_3_Shots(rec, 'gpt4', 0, 60), axis=1)
sample_test_df['gpt4-ets-3_shots_med_temp_60_words'] = sample_test_df.apply(lambda rec: generate_GPT_ETS_3_Shots(rec, 'gpt4', 1, 60), axis=1)
sample_test_df['gpt4-ets-3_shots_high_temp_60_words'] = sample_test_df.apply(lambda rec: generate_GPT_ETS_3_Shots(rec, 'gpt4', 2, 60), axis=1)

sample_test_df['gpt35-ets-3_shots_low_temp_60_words'] = sample_test_df.apply(lambda rec: generate_GPT_ETS_3_Shots(rec, 'gpt35', 0, 60), axis=1)
sample_test_df['gpt35-ets-3_shots_med_temp_60_words'] = sample_test_df.apply(lambda rec: generate_GPT_ETS_3_Shots(rec, 'gpt35', 1, 60), axis=1)
sample_test_df['gpt35-ets-3_shots_high_temp_60_words'] = sample_test_df.apply(lambda rec: generate_GPT_ETS_3_Shots(rec, 'gpt35', 2, 60), axis=1)

sample_test_df['gpt4-ets-1_shot_low_temp_60_words'] = sample_test_df.apply(lambda rec: generate_GPT_ETS_1_Shot(rec, 'gpt4', 0, 60), axis=1)
sample_test_df['gpt4-ets-1_shot_med_temp_60_words'] = sample_test_df.apply(lambda rec: generate_GPT_ETS_1_Shot(rec, 'gpt4', 1, 60), axis=1)
sample_test_df['gpt4-ets-1_shot_high_temp_60_words'] = sample_test_df.apply(lambda rec: generate_GPT_ETS_1_Shot(rec, 'gpt4', 2, 60), axis=1)

sample_test_df['gpt35-ets-1_shot_low_temp_60_words'] = sample_test_df.apply(lambda rec: generate_GPT_ETS_1_Shot(rec, 'gpt35', 0, 60), axis=1)
sample_test_df['gpt35-ets-1_shot_med_temp_60_words'] = sample_test_df.apply(lambda rec: generate_GPT_ETS_1_Shot(rec, 'gpt35', 1, 60), axis=1)
sample_test_df['gpt35-ets-1_shot_high_temp_60_words'] = sample_test_df.apply(lambda rec: generate_GPT_ETS_1_Shot(rec, 'gpt35', 2, 60), axis=1)


Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': True, 'severity': 'medium'}}}}}
Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param

In [32]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
models = [

          'gpt4-ets-5_shots_low_temp_60_words', 
          'gpt4-ets-5_shots_med_temp_60_words' , 
          'gpt4-ets-5_shots_high_temp_60_words',
          'gpt35-ets-5_shots_low_temp_60_words', 
          'gpt35-ets-5_shots_med_temp_60_words' , 
          'gpt35-ets-5_shots_high_temp_60_words',

          'gpt4-ets-3_shots_low_temp_60_words', 
          'gpt4-ets-3_shots_med_temp_60_words' , 
          'gpt4-ets-3_shots_high_temp_60_words',
          'gpt35-ets-3_shots_low_temp_60_words', 
          'gpt35-ets-3_shots_med_temp_60_words' , 
          'gpt35-ets-3_shots_high_temp_60_words',
    
          'gpt4-ets-1_shot_low_temp_60_words', 
          'gpt4-ets-1_shot_med_temp_60_words' , 
          'gpt4-ets-1_shot_high_temp_60_words',
          'gpt35-ets-1_shot_low_temp_60_words', 
          'gpt35-ets-1_shot_med_temp_60_words' , 
          'gpt35-ets-1_shot_high_temp_60_words',
         ]

records = []

for model_name in models:
    rouge_metric.add(prediction = sample_test_df[model_name], reference = sample_test_df['highlights'])
    score = rouge_metric.compute()
    rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )
    print('rouge_dict ', rouge_dict )
    records.append(rouge_dict)

metrics_df = pd.DataFrame.from_records(records, index = models )
metrics_df

rouge_dict  {'rouge1': 0.4980147475893364, 'rouge2': 0.17603634298693924, 'rougeL': 0.24503686897334087, 'rougeLsum': 0.24503686897334087}
rouge_dict  {'rouge1': 0.4766355140186916, 'rouge2': 0.15789473684210523, 'rougeL': 0.2336448598130841, 'rougeLsum': 0.2336448598130841}
rouge_dict  {'rouge1': 0.49144811858608894, 'rouge2': 0.17579908675799086, 'rougeL': 0.2371721778791334, 'rougeLsum': 0.2371721778791334}
rouge_dict  {'rouge1': 0.5016987542468856, 'rouge2': 0.1598639455782313, 'rougeL': 0.23895809739524349, 'rougeLsum': 0.23895809739524349}
rouge_dict  {'rouge1': 0.4923413566739606, 'rouge2': 0.1325301204819277, 'rougeL': 0.22975929978118162, 'rougeLsum': 0.22975929978118162}
rouge_dict  {'rouge1': 0.47287264420331243, 'rouge2': 0.12464265294453973, 'rougeL': 0.20902341519131926, 'rougeLsum': 0.20902341519131926}
rouge_dict  {'rouge1': 0.49885583524027455, 'rouge2': 0.15693012600229095, 'rougeL': 0.2322654462242563, 'rougeLsum': 0.2322654462242563}
rouge_dict  {'rouge1': 0.4923076

,rouge1,rouge2,rougeL,rougeLsum
gpt4-ets-5_shots_low_temp_60_words,0.498015,0.176036,0.245037,0.245037
gpt4-ets-5_shots_med_temp_60_words,0.476636,0.157895,0.233645,0.233645
gpt4-ets-5_shots_high_temp_60_words,0.491448,0.175799,0.237172,0.237172
gpt35-ets-5_shots_low_temp_60_words,0.501699,0.159864,0.238958,0.238958
gpt35-ets-5_shots_med_temp_60_words,0.492341,0.132530,0.229759,0.229759
gpt35-ets-5_shots_high_temp_60_words,0.472873,0.124643,0.209023,0.209023
gpt4-ets-3_shots_low_temp_60_words,0.498856,0.156930,0.232265,0.232265
gpt4-ets-3_shots_med_temp_60_words,0.492308,0.158768,0.235503,0.235503
gpt4-ets-3_shots_high_temp_60_words,0.440936,0.163934,0.219883,0.219883
gpt35-ets-3_shots_low_temp_60_words,0.527072,0.181416,0.251934,0.251934


In [34]:
metrics_df

,rouge1,rouge2,rougeL,rougeLsum
gpt4-ets-5_shots_low_temp_60_words,0.498015,0.176036,0.245037,0.245037
gpt4-ets-5_shots_med_temp_60_words,0.476636,0.157895,0.233645,0.233645
gpt4-ets-5_shots_high_temp_60_words,0.491448,0.175799,0.237172,0.237172
gpt35-ets-5_shots_low_temp_60_words,0.501699,0.159864,0.238958,0.238958
gpt35-ets-5_shots_med_temp_60_words,0.492341,0.132530,0.229759,0.229759
gpt35-ets-5_shots_high_temp_60_words,0.472873,0.124643,0.209023,0.209023
gpt4-ets-3_shots_low_temp_60_words,0.498856,0.156930,0.232265,0.232265
gpt4-ets-3_shots_med_temp_60_words,0.492308,0.158768,0.235503,0.235503
gpt4-ets-3_shots_high_temp_60_words,0.440936,0.163934,0.219883,0.219883
gpt35-ets-3_shots_low_temp_60_words,0.527072,0.181416,0.251934,0.251934


In [33]:
metrics_df.to_csv("./output/GPT_ETS_FS_Metrics.csv")